In [3]:
pip install surprise

  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-win_amd64.whl size=707142 sha256=a4822d777e0479994efa25819428bc21cd57ec91427a8493b102339c014775ad
  Stored in directory: c:\users\aazam\appdata\local\pip\cache\wheels\20\91\57\2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install implicit

  Created wheel for implicit: filename=implicit-0.4.4-cp38-cp38-win_amd64.whl size=613980 sha256=1684e6e9c7117ee3925896bdf86315f92ebc359cab93969f143379a033b9e124
  Stored in directory: c:\users\aazam\appdata\local\pip\cache\wheels\00\ac\67\6f4536c819ed560c2c7e17c0f7a920e3e50c26108616087d05
Successfully built implicit



In [34]:
import pandas as pd
import numpy as np
import re

from surprise import KNNBasic, SlopeOne, BaselineOnly, SVD,KNNWithMeans, KNNBaseline
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import KFold

import scipy.sparse as sparse
from implicit.als import AlternatingLeastSquares

from collections import defaultdict

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

from warnings import filterwarnings 
filterwarnings('ignore')

Считываем данные о фильмах, рейтингах и тэгах

In [5]:
movies  = pd.read_csv('D:/Учеба/Lern DS of Netology/Курс ML 2 Рекомендательные системы/12_Введение и классификация рекомендательных систем/movies.csv')
ratings = pd.read_csv('D:/Учеба/Lern DS of Netology/Курс ML 2 Рекомендательные системы/12_Введение и классификация рекомендательных систем/ratings.csv')
tags    = pd.read_csv('D:/Учеба/Lern DS of Netology/Курс ML 2 Рекомендательные системы/12_Введение и классификация рекомендательных систем/tags.csv')
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703


Из состава выборки рекомендуемых фильмов исключаем средние и плохие фильмы (медианный рейтинг меньше 4)

In [7]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [9]:
movies_with_ratings.head(5)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [19]:
tags.head(5)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [25]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [28]:
movie_genres = [change_string(g) for g in movies.genres.values]
movie_genres[0:5]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy']

In [23]:
title_genres = {}

for index, row in movies.iterrows():
    title_genres[row.title] = row.genres

In [32]:
#title_genres

In [35]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

neigh = NearestNeighbors(n_neighbors=20, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=20)

In [37]:
def recommend_for_user(user_id):
    current_user_id = user_id
    user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()
    
    last_user_movie = user_movies[-1]
    
    movie_genres = title_genres[last_user_movie]
    
    movie_genres = change_string(movie_genres)

    predict = count_vect.transform([movie_genres])
    X_tfidf2 = tfidf_transformer.transform(predict)

    res = neigh.kneighbors(X_tfidf2, return_distance=True)
    
    movies_to_score = movies.iloc[res[1][0]].title.values

    scores = []
    titles = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue

        scores.append(algo.predict(uid=current_user_id, iid=movie).est)
        titles.append(movie)
        
    
    best_indexes = np.argsort(scores)[-5:]
    for i in reversed(best_indexes):
        print(titles[i], scores[i])

In [39]:
recommend_for_user(2)

Samsara (2011) 3.7536814168093544
My Kid Could Paint That (2007) 3.7336515639184107
Tomorrow (2015) 3.705953309702564
Nobody Speak: Hulk Hogan, Gawker and Trials of a Free Press (2017) 3.683359524764664
Comandante (2003) 3.667959715331909
